In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from glob import glob
import cv2

from keras.utils import to_categorical

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = []
train_labels = []

for i in glob("../input/seefood/train/*/*"):
    aux = i.split('/')[4]
    img = cv2.imread(i)[:,:,::-1]
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, 0)
    
    train_data.append(img)
    if aux == 'not_hot_dog':
        train_labels.append(0)
    else:
        train_labels.append(1)

In [ ]:
test_data = []
test_labels = []

for i in glob("../input/seefood/test/*/*"):
    aux = i.split('/')[4]
    img = cv2.imread(i)[:,:,::-1]
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, 0)
    
    test_data.append(img)
    if aux == 'not_hot_dog':
        test_labels.append(0)
    else:
        test_labels.append(1)

In [ ]:
train_data = np.vstack(train_data)
test_data = np.vstack(test_data)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
train_data.shape, test_data.shape

<h2>Visualização</h2>

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.imshow(train_data[1]), print(train_labels[1])

In [ ]:
np.count_nonzero(train_labels == 1), np.count_nonzero(train_labels == 0)

<h2>Data Generator</h2>

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [ ]:
datagen.fit(train_data)

<h2>Normalização</h2>

In [ ]:
from __future__ import division

In [ ]:
train_data = train_data / 255
test_data = test_data / 255

<h2>Modelo CNN</h2>

In [ ]:
from keras.models import Model
from keras.layers import *
from keras.applications import MobileNet

In [ ]:
model = MobileNet(input_shape=(224, 224, 3), include_top=True, classes=2, weights=None)

In [ ]:
model.summary()

In [ ]:
# input_node = Input(shape=(224, 224, 3))
# cnn1 = Conv2D(32, (3, 3), padding='same', activation='relu')(input_node)
# pool1 = MaxPooling2D((2, 2), strides=(2, 2))(cnn1)

# cnn2 = Conv2D(64, (3, 3), padding='same', activation='relu')(pool1)
# pool2 = MaxPooling2D((2, 2), strides=(2, 2))(cnn2)

# cnn3 = Conv2D(128, (3, 3), padding='same', activation='relu')(pool2)
# pool3 = MaxPooling2D((2, 2), strides=(2, 2))(cnn3)

# cnn4 = Conv2D(256, (3, 3), padding='same', activation='relu')(pool3)
# pool4 = MaxPooling2D((2, 2), strides=(2, 2))(cnn4)

# cnn5 = Conv2D(512, (3, 3), padding='same', activation='relu')(pool4)
# pool5 = MaxPooling2D((2, 2), strides=(2, 2))(cnn5)

# cnn6 = Conv2D(1024, (3, 3), padding='same', activation='relu')(pool5)
# pool6 = MaxPooling2D((2, 2), strides=(2, 2))(cnn6)

# flat = Flatten()(pool6)
# fc1 = Dense(1024)(flat)
# fc2 = Dense(1024)(fc1)
# output_node = Dense(1, activation='sigmoid')(fc2)

# model = Model(input_node, output_node)
# # model.summary()
# len(model.layers)

In [ ]:
# model.summary()

In [ ]:
from keras.optimizers import *
from keras.metrics import *
from keras.callbacks import *

In [ ]:
model.compile('Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
folder = 'logs/'

if not os.path.isdir(folder):
    os.makedirs(folder)

model_checkpoint = ModelCheckpoint(filepath=folder + 'model.{epoch:02d}-{loss:.2f}-{acc:.2f}-{val_loss:.2f}-{val_acc:.2f}.hdf5', 
                                   verbose=1, 
                                   save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss', patience=20)

callbacks = [early_stop, model_checkpoint]

In [ ]:
epochs = 1000
model.fit_generator(datagen.flow(train_data, train_labels, batch_size=8),
                    steps_per_epoch=len(train_data) / 8, epochs=epochs, callbacks = callbacks, validation_data=datagen.flow(test_data, test_labels, batch_size=8))

In [ ]:
model.evaluate_generator(datagen.flow(test_data, test_labels, batch_size=8))